In [20]:
# DASHBOARD 8015
import dash
from dash import html, dcc, Input, Output
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Load data
df = pd.read_csv("df_final.csv", parse_dates=["saledate"])
df_unique = df.drop_duplicates(subset="orderid").reset_index(drop=True)

# Filters
districts = sorted(df_unique['Quận/Huyện'].dropna().unique())
time_options = df_unique['Time of day'].dropna().unique()
weekday_options = df_unique['Weekday'].dropna().unique()
age_groups = df_unique['Age Group'].dropna().unique()

# App
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("🎬 Dashboard Doanh Thu Và Vé Bán Của Rạp Phim", style={
        'textAlign': 'center',
        'color': '#003049',
        'marginBottom': '20px'
    }),

    html.Div([
        html.Div([
            html.Label("Chọn Quận/Huyện:"),
            dcc.Dropdown(
                options=[{'label': d, 'value': d} for d in districts],
                id='district-filter', value=None, placeholder="Tất cả"
            )
        ], style={'width': '24%', 'display': 'inline-block', 'paddingRight': '10px'}),

        html.Div([
            html.Label("Chọn Thời gian trong ngày:"),
            dcc.Dropdown(
                options=[{'label': t, 'value': t} for t in time_options],
                id='time-filter', value=None, placeholder="Tất cả"
            )
        ], style={'width': '24%', 'display': 'inline-block', 'paddingRight': '10px'}),

        html.Div([
            html.Label("Chọn Thứ trong tuần:"),
            dcc.Dropdown(
                options=[{'label': w, 'value': w} for w in weekday_options],
                id='weekday-filter', value=None, placeholder="Tất cả"
            )
        ], style={'width': '24%', 'display': 'inline-block', 'paddingRight': '10px'}),

        html.Div([
            html.Label("Chọn Nhóm tuổi:"),
            dcc.Dropdown(
                options=[{'label': a, 'value': a} for a in age_groups],
                id='age-filter', value=None, placeholder="Tất cả"
            )
        ], style={'width': '24%', 'display': 'inline-block'})
    ], style={'display': 'flex', 'flexWrap': 'wrap', 'marginBottom': '30px'}),

    # Hàng 1
    html.Div([
        html.Div(dcc.Graph(id='chart4', figure=px.bar(
            df_unique.groupby('Weekday')['total'].sum().reset_index().sort_values(by='total', ascending=False),
            x='Weekday', y='total',
            title='Doanh Thu Theo Ngày Trong Tuần'
        )), style={'width': '32%', 'display': 'inline-block', 'paddingRight': '1%'}),

        html.Div(dcc.Graph(id='chart2', figure=px.bar(
            df_unique, x='Time of day', y='total',
            title='Doanh thu theo thời điểm trong ngày',
            labels={'total': 'Tổng doanh thu', 'Time of day': 'Thời điểm'},
            color='Time of day'
        )), style={'width': '32%', 'display': 'inline-block', 'paddingRight': '1%'}),

        html.Div(dcc.Graph(id='chart3', figure=px.line(
            df_unique[df_unique['saledate'].dt.month == 5]
            .groupby(df_unique['saledate'].dt.day)['total'].sum().reset_index(),
            x='saledate', y='total', markers=True,
            labels={'saledate': 'Ngày', 'total': 'Tổng doanh thu'},
            title='Tổng doanh thu theo ngày trong tháng 5'
        )), style={'width': '32%', 'display': 'inline-block'})
    ], style={'display': 'flex', 'flexWrap': 'wrap', 'marginBottom': '30px'}),

    # Hàng 2
    html.Div([
        html.Div(dcc.Graph(id='chart5'), style={'width': '100%', 'display': 'inline-block'})
    ], style={'display': 'flex', 'flexWrap': 'wrap'}),

    # Hàng 3
    html.Div([
        html.Div([dcc.Graph(id='chart6', figure=px.bar(
            df_unique.groupby(['Age Group', 'Gender'])['total'].sum().reset_index(),
            x='Age Group', y='total', color='Gender', barmode='group',
            title='Doanh Thu Theo Nhóm Tuổi và Giới Tính', color_discrete_map={
                'Nam': '#8E1616',
                'Nữ': '#3A59D1'
            }))], style={'width': '33%', 'display': 'inline-block'}),

        html.Div([dcc.Graph(id='chart7', figure=px.pie(
            df.explode('country').groupby('country')['total'].sum().reset_index(),
            names='country', values='total',
            title='Doanh Thu Theo Quốc Gia Phim', color_discrete_map={
                'United States': '#3A59D1',
                'Vietnam': '#8E1616',
                'Japan': '#7AC6D2',
                'Thailand': '#E8C999',
                'Korea': '#BF9264',
                'Belgium': '#f8961e'
            }))], style={'width': '33%', 'display': 'inline-block'}),

        html.Div([dcc.Graph(id='chart8', figure=px.bar(
            df.groupby('listed_in')['total'].sum().reset_index().sort_values(by='total', ascending=False),
            y='listed_in', x='total', orientation='h',
            title='Doanh Thu Theo Thể Loại Phim'
        ))], style={'width': '33%', 'display': 'inline-block'})
    ], style={'display': 'flex', 'flexWrap': 'wrap'})
])

@app.callback(
    Output('chart5', 'figure'),
    [Input('time-filter', 'value'),
     Input('age-filter', 'value'),
     Input('weekday-filter', 'value')]
)
def update_chart5(selected_time, selected_age, selected_weekday):
    df_plot = df_unique.copy()
    if selected_time:
        df_plot = df_plot[df_plot['Time of day'] == selected_time]
    if selected_age:
        df_plot = df_plot[df_plot['Age Group'] == selected_age]
    if selected_weekday:
        df_plot = df_plot[df_plot['Weekday'] == selected_weekday]

    df_plot['hour'] = pd.to_datetime(df_plot['time']).dt.hour
    hour_counts = df_plot['hour'].value_counts().sort_index()
    top3_hours = hour_counts.sort_values(ascending=False).head(3).index

    bar_trace = go.Bar(
        x=hour_counts.index,
        y=hour_counts.values,
        marker_color=['maroon' if hour in top3_hours else '#669BBC' for hour in hour_counts.index],
        name='Số lượt đặt vé',
        text=hour_counts.values,
        textposition='outside'
    )

    line_trace = go.Scatter(
        x=hour_counts.index,
        y=hour_counts.values,
        mode='lines+markers',
        line=dict(color='#D29F80', width=3),
        name='Xu hướng'
    )

    fig = go.Figure(data=[bar_trace, line_trace])
    fig.update_layout(
        title='Phân bố số lượng khách hàng theo giờ trong ngày',
        xaxis_title='Giờ trong ngày',
        yaxis_title='Số lượt đặt vé',
        legend_title='Chú thích',
        bargap=0.2,
        template='simple_white',
        xaxis=dict(dtick=1)
    )
    return fig

if __name__ == '__main__':
    app.run(debug=True, port=8015)